<a href="https://colab.research.google.com/github/chayan141/Langgraph-Projects/blob/main/Langgraph_Long_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Long Term Memory General

Semantic Memory : Facts about the users.

Episodic Memory : Involves recalling Past Experience or events or acions. It is sometimes implemented through the fewshot prompting.

Procedural Memory : Both humans and AI agents remembers set of rules to perform any tasks.

In [1]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph trustcall langchain_core langchain-google-genai

In [2]:
from google.colab import userdata
gemini = userdata.get('gemini_api_key')
import os
os.environ['gemini_api_key'] = gemini

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash',api_key=gemini)

In [5]:
from langchain_core.runnables import RunnableConfig
from langgraph.config import get_store
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore

In [6]:
store = InMemoryStore()

In [8]:
# TypedDict instance
user_profile = {
    "user_name": "Lance",
    "interests": ["biking", "technology", "coffee"]
}
user_profile

{'user_name': 'Lance', 'interests': ['biking', 'technology', 'coffee']}

In [9]:
namespace_for_memory = ("1", "memory")

key = "user_profile"
value = user_profile

In [10]:
store.put(namespace_for_memory, key, value)

In [12]:
for m in store.search(namespace_for_memory):
    print(m.dict())

{'namespace': ['1', 'memory'], 'key': 'user_profile', 'value': {'user_name': 'Lance', 'interests': ['biking', 'technology', 'coffee']}, 'created_at': '2025-07-04T03:06:57.182191+00:00', 'updated_at': '2025-07-04T03:06:57.182196+00:00', 'score': None}


In [13]:
type(m)

langgraph.store.base.SearchItem

In [14]:
profile = store.get(namespace_for_memory, key)
profile.value

{'user_name': 'Lance', 'interests': ['biking', 'technology', 'coffee']}

# Read Long Term Memory

In [18]:
def get_user_info(config: RunnableConfig):
  """Look up user info."""
    # Same as that provided to `create_react_agent`
  store = get_store()
  user_id = config['configurable'].get("user_id")
  user_info = store.get(("1",  "memory"), user_id)
  return user_info.value if user_info else "Unknown Users"

In [19]:
agent = create_react_agent(
    model = model,
    tools = [get_user_info],
    store = store
)

In [20]:
agent.invoke(
    {"messages": [{"role": "user", "content": "look up user information"}]},
    config={"configurable": {"user_id": "1"}}
)

{'messages': [HumanMessage(content='look up user information', additional_kwargs={}, response_metadata={}, id='67009cc5-69a8-4aa4-88b7-414982837d87'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_user_info', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0a3574db-a75c-42b9-a751-fa02f90d12d9-0', tool_calls=[{'name': 'get_user_info', 'args': {}, 'id': 'ce3fab0f-eb7d-49dd-936d-7a910fc48025', 'type': 'tool_call'}], usage_metadata={'input_tokens': 14, 'output_tokens': 5, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='Unknown Users', name='get_user_info', id='369df786-8572-45ac-b478-bc8895bc8f63', tool_call_id='ce3fab0f-eb7d-49dd-936d-7a910fc48025'),
  AIMessage(content='I looked up the user information and the result is "Unknown Users".', additional_kwargs={}, response_metadata={

# Write Long Term Memory

In [25]:
from typing_extensions import TypedDict
from typing import List

from langgraph.config import get_store
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()

class UserProfile(TypedDict):
    """User profile schema with typed fields"""
    user_name: str  # The user's preferred name
    interests: List[str]  # A list of the user's interests

In [26]:
def save_user_info(user_info: UserProfile, config: RunnableConfig):
    """Save user info."""

    store = get_store()
    user_id = config['configurable'].get("user_id")
    store.put(("2", "memory"), user_id, user_info)
    return "Success"

In [27]:
agent = create_react_agent(
    model = model,
    tools = [save_user_info],
    store = store
)

In [28]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "My name is John Smith, I have interests in bike riding"}]},
    config={"configurable": {"user_id": "user_123"}}
)

{'messages': [HumanMessage(content='My name is John Smith, I have interests in bike riding', additional_kwargs={}, response_metadata={}, id='09409924-a871-4fd2-8cc5-45fe42e99fd4'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'save_user_info', 'arguments': '{"user_info": {"interests": ["bike riding"], "user_name": "John Smith"}}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--567b2f99-9f5e-4823-aaae-08eb57ab4989-0', tool_calls=[{'name': 'save_user_info', 'args': {'user_info': {'interests': ['bike riding'], 'user_name': 'John Smith'}}, 'id': '94c0065c-a7d1-4644-accb-7f70ff35bc64', 'type': 'tool_call'}], usage_metadata={'input_tokens': 46, 'output_tokens': 16, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='Success', name='save_user_info', id='fc9fa25a-523a-41bf-b6f0-3bfe5ef54052', tool_call_id='94c0

In [29]:
namespace_for_memory = ("2", "memory")
for m in store.search(namespace_for_memory):
    print(m.dict())

{'namespace': ['2', 'memory'], 'key': 'user_123', 'value': {'user_name': 'John Smith', 'interests': ['bike riding']}, 'created_at': '2025-07-04T03:27:16.376406+00:00', 'updated_at': '2025-07-04T03:27:16.376410+00:00', 'score': None}


# Complex Schema Processing Using TrustCall

Complex schemas can be difficult to extract.

In addition, updating even simple schemas can pose challenges.

Consider our above chatbot.

We regenerated the profile schema from scratch each time we chose to save a new memory.

This is inefficient, potentially wasting model tokens if the schema contains a lot of information to re-generate each time.

Worse, we may loose information when regenerating the profile from scratch.

Addressing these problems is the motivation for TrustCall!

In [30]:

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
# Conversation
conversation = [HumanMessage(content="Hi, I'm Lance."),
                AIMessage(content="Nice to meet you, Lance."),
                HumanMessage(content="I really like biking around San Francisco.")]

In [31]:
from trustcall import create_extractor
from pydantic import BaseModel, Field



# Schema
class UserProfile(BaseModel):
    """User profile schema with typed fields"""
    user_name: str = Field(description="The user's preferred name")
    interests: List[str] = Field(description="A list of the user's interests")

In [32]:
# Create the extractor
trustcall_extractor = create_extractor(
    model,
    tools=[UserProfile],
    tool_choice="UserProfile"
)

In [33]:
# Instruction
system_msg = "Extract the user profile from the following conversation"

# Invoke the extractor
result = trustcall_extractor.invoke({"messages": [SystemMessage(content=system_msg)]+conversation})

In [34]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  UserProfile (1d6851df-f4dd-4eda-bcd6-01c1e81e8f17)
 Call ID: 1d6851df-f4dd-4eda-bcd6-01c1e81e8f17
  Args:
    interests: ['biking']
    user_name: Lance


In [35]:
schema = result["responses"]
schema

[UserProfile(user_name='Lance', interests=['biking'])]

In [36]:

schema[0].model_dump()

{'user_name': 'Lance', 'interests': ['biking']}

In [37]:
result["response_metadata"]

[{'id': '1d6851df-f4dd-4eda-bcd6-01c1e81e8f17'}]

In [38]:
# Update the conversation
updated_conversation = [HumanMessage(content="Hi, I'm Lance."),
                        AIMessage(content="Nice to meet you, Lance."),
                        HumanMessage(content="I really like biking around San Francisco."),
                        AIMessage(content="San Francisco is a great city! Where do you go after biking?"),
                        HumanMessage(content="I really like to go to a bakery after biking."),]

In [39]:
# Update the instruction
system_msg = f"""Update the memory (JSON doc) to incorporate new information from the following conversation"""

In [40]:
# Invoke the extractor with the updated instruction and existing profile with the corresponding tool name (UserProfile)
result = trustcall_extractor.invoke({"messages": [SystemMessage(content=system_msg)]+updated_conversation},
                                    {"existing": {"UserProfile": schema[0].model_dump()}})

for m in result["messages"]:
    m.pretty_print()


================================== Ai Message ==================================
Tool Calls:
  UserProfile (3e39644d-6a29-4567-89a3-959f96d208b7)
 Call ID: 3e39644d-6a29-4567-89a3-959f96d208b7
  Args:
    interests: ['biking', 'bakery']
    user_name: Lance


In [41]:
result["response_metadata"]

[{'id': '3e39644d-6a29-4567-89a3-959f96d208b7'}]

In [42]:
updated_schema = result["responses"][0]
updated_schema.model_dump()

{'user_name': 'Lance', 'interests': ['biking', 'bakery']}

# Managing Short Term Memory

With short-term memory enabled, long conversations can exceed the LLM's context window. Common solutions are:

1. Trim Messages : Remove first or last N messages (before calling an LLM).

2. Delete Messages from langgraph state permanantly.

3. Summarize Messages : Summarize earlier messages in the history and replace them with a summary.

4. Manage Checkpoints to store and retrieve message history.

Trim Messages : To trim message history in an agent, use pre_model_hook with the trim_messages function

In [50]:
from langchain_core.messages.utils import trim_messages, count_tokens_approximately
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.checkpoint.memory import InMemorySaver

summarization_model = model.bind(max_tokens=128)

def call_model(state: MessagesState):
  messages = trim_messages(
      state['messages'],
      strategy="last",
      token_counter = count_tokens_approximately,
      max_tokens=128,
      start_on = "human",
      end_on = ("human","tool")
  )

  response = model.invoke(messages)
  return {"messages":[response]}

In [51]:
checkpointer = InMemorySaver()
builder = StateGraph(MessagesState)
builder.add_node("call_model",call_model)
builder.add_edge(START, "call_model")
graph = builder.compile(checkpointer=checkpointer)

In [52]:
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": "hi, my name is bob"}, config)
graph.invoke({"messages": "write a short poem about cats"}, config)
graph.invoke({"messages": "now do the same but for dogs"}, config)
final_response = graph.invoke({"messages": "what's my name?"}, config)

final_response["messages"][-1].pretty_print()

================================== Ai Message ==================================

As a large language model, I don't have access to your personal information, including your name. You haven't told me your name, so I don't know it.


# Summarize Messages

In [53]:
!pip install langmem --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 12.0 MB/s eta 0:00:00


In [66]:
from langmem.short_term import SummarizationNode
from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from typing import Any
from langchain_core.messages import AnyMessage

We will keep track of our running summary in the context field (expected by the SummarizationNode).

In [67]:
class State(MessagesState):
    context: dict[str, Any]

Define private state that will be used only for filtering the inputs to call_model node.

In [68]:
class LLMInputState(TypedDict):
    summarized_messages: list[AnyMessage]
    context: dict[str, Any]


In [69]:
summarization_node = SummarizationNode(
    token_counter=count_tokens_approximately,
    model=summarization_model,
    max_tokens=256,
    max_tokens_before_summary=256,
    max_summary_tokens=128,
)

In [70]:
def call_model(state: LLMInputState):
    response = model.invoke(state["summarized_messages"])
    return {"messages": [response]}

In [71]:
checkpointer = InMemorySaver()
builder = StateGraph(State)
builder.add_node(call_model)
builder.add_node("summarize", summarization_node)
builder.add_edge(START, "summarize")
builder.add_edge("summarize", "call_model")
graph = builder.compile(checkpointer=checkpointer)

In [72]:
# Invoke the graph
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": "hi, my name is bob"}, config)
graph.invoke({"messages": "write a short poem about cats"}, config)
graph.invoke({"messages": "now do the same but for dogs"}, config)
final_response = graph.invoke({"messages": "what's my name?"}, config)

In [75]:
final_response["messages"][-1].pretty_print()
final_response

================================== Ai Message ==================================

Your name is Bob. You told me at the beginning of our conversation.


{'messages': [HumanMessage(content='hi, my name is bob', additional_kwargs={}, response_metadata={}, id='d522eca2-c932-4e21-93aa-92224965136e'),
  AIMessage(content="Hi Bob, it's nice to meet you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c0f66858-df80-40fe-9189-b99944dcf430-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='write a short poem about cats', additional_kwargs={}, response_metadata={}, id='f2fb6495-ebc5-4838-b860-e53705230778'),
  AIMessage(content='With eyes of emerald, gold, or blue,\nA furry friend, both sleek and true.\nA silent hunter, soft and sly,\nA purring rumble, passing by.\n\nA graceful leap, a playful swat,\nA sunbeam nap, upon the mat.\nA curious gaze, a twitching tail,\nA feline charm 

In [76]:
for m in final_response["messages"]:
    m.pretty_print()

================================ Human Message =================================

hi, my name is bob
================================== Ai Message ==================================

Hi Bob, it's nice to meet you! How can I help you today?
================================ Human Message =================================

write a short poem about cats
================================== Ai Message ==================================

With eyes of emerald, gold, or blue,
A furry friend, both sleek and true.
A silent hunter, soft and sly,
A purring rumble, passing by.

A graceful leap, a playful swat,
A sunbeam nap, upon the mat.
A curious gaze, a twitching tail,
A feline charm that will not fail.
================================ Human Message =================================

now do the same but for dogs
================================== Ai Message ==================================

With wagging tail and happy bark,
A loyal friend, both light and dark.
A furry pal, with boundless glee,
A

# Context

Context Engineering is the practice of building dynamic systems that provide the right information and tools in the right format, so that a language model can plausibly accomplish a task.

config : data passed at the start of a run.

short_term_memory(State)

long_term_memory(Store)

# Human In The Loop

Approve or reject: Pause the graph before a critical step.

Edit Graph State: Pause the graph to review and edit the graph state.

Reviewing tool calls and validate human input.